In [ ]:
!pip install langchain chromadb wikipedia-api

In [ ]:
!pip install -U bitsandbytes langchain-community

Import libaries

In [3]:
import chromadb
import wikipediaapi
from chromadb.config import Settings
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, set_seed

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

from langchain import PromptTemplate, LLMChain
from langchain.chains import StuffDocumentsChain
from langchain.docstore.document import Document
import os

setup chromadb

In [4]:
chroma_client = chromadb.PersistentClient(path="db2/")

wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent='extract data'
)

collection = chroma_client.get_or_create_collection(name="snd_collection")

article_titles = ["machine learning"]

extract articles from wikipedia

In [5]:
# extract text from wikipedia
def extract_text_from_wikipedia(title):
    page = wiki_wiki.page(title)
    if page.exists():
        return page.text
    else:
        print(f"Article '{title}' does not exist on Wikipedia.")
        return None

In [6]:
# add articles to chromadb
for title in article_titles:
    article_text = extract_text_from_wikipedia(title)

    if article_text:
      collection.upsert(
            documents=[article_text],
            ids=[title])

In [7]:
# add new articles to existing chromadb
new_articles = ['software engineering']
for title in new_articles:
    article_text = extract_text_from_wikipedia(title)

    if article_text:
      collection.upsert(
            documents=[article_text],
            ids=[title])

In [ ]:
chroma_client = chromadb.PersistentClient(path="db2/")
collection = chroma_client.get_or_create_collection(name="snd_collection")

# get all ids from the collection
ids = collection.get(ids=collection.get()['ids'])['ids']

# print all ids
ids

download phi model

In [ ]:
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

In [26]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
llm = HuggingFacePipeline(pipeline=pipe)

In [11]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

final prompt teemplate creation and RAG implementation

In [27]:
def phi_query_chroma_and_generate_response(query, chroma_collection, top_k=2, max_context_length=20000):
    results = chroma_collection.query(query_texts=[query], n_results=top_k)
    context = "\n".join(results['documents'][0])

    # truncate context if it exceeds max_context_length -- this was done to fine out of memory error in GPU
    if len(context) > max_context_length:
        context = context[:max_context_length]

     system_prompt = "<|system|> Use the following context to answer the question.<|end|>"
    user_prompt = f"<|user|> {query}<|end|>"
    full_prompt = f"{system_prompt}\n<|context|> {{context}}<|end|>\n{user_prompt}<|assistant|>"

    PROMPT = PromptTemplate(template=full_prompt, input_variables=["context", "query"])
    llm_chain = LLMChain(llm=llm, prompt=PROMPT)

    chain = StuffDocumentsChain(
        llm_chain=llm_chain,
        document_variable_name="context",
    )

    answer = (chain({"input_documents": [Document(page_content=context)], "query": query}, return_only_outputs=True))['output_text']
    answer = (answer.split("<|assistant|>")[-1]).strip()
    return answer

In [ ]:
# usage example
user_query = "what is machine learning?"
response = phi_query_chroma_and_generate_response(user_query, collection)
print(response)

In [ ]:
# usage examples
user_query = "what is software engineering?"
response = phi_query_chroma_and_generate_response(user_query, collection)
print(response)

In [ ]:
# Example usag
user_query = "what is difference between software engineering and machine learning?"
response = phi_query_chroma_and_generate_response(user_query, collection)
print(response)